In [1]:
import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Definir la ruta del dataset (dentro de la carpeta training)
base_dir = '/home/adrian/Desktop/Aprendizaje/practica_final/training'
dataset_path = os.path.join(base_dir, 'dataset')

# Aseguramos que la estructura sea la correcta (train/images, valid/images)
# Esta estructura ya existe en tu carpeta training/dataset
print(f"Estructura de dataset localizada en: {dataset_path}")


Estructura de dataset localizada en: /home/adrian/Desktop/Aprendizaje/practica_final/training/dataset


# Aplicación de Detección y Segmentación de Productos con YOLO (Transfer Learning)

Este notebook contiene el flujo de trabajo para entrenar un modelo YOLO para la detección y segmentación de productos basados en SKU específicos.

## 1. Configuración del Entorno y Datos

Para YOLO, necesitamos una estructura de carpetas específica:
```
dataset/
├── images/
│   ├── train/
│   └── val/
└── labels/
    ├── train/
    └── val/
```
Cada producto (SKU) se mapeará a un ID de clase.


## 2. Creación del archivo de configuración `data.yaml`

Este archivo le indica a YOLO dónde encontrar las imágenes y cuáles son las etiquetas (SKUs).


In [2]:
import yaml

# SKUs encontrados en el dataset
skus = {
    0: 'SKU-LAPTOP-let01',
    1: 'SKU-LAPTOP-al01',
    2: 'SKU-LAPTOP-asu01',    
    3: 'SKU-LAPTOP-mbk01',
    
}

data_config = {
    'path': dataset_path,
    'train': 'train/images',
    'val': 'valid/images',
    'names': skus
}

# Guardamos el archivo data.yaml dentro de la carpeta del dataset
yaml_file_path = os.path.join(dataset_path, 'data.yaml')
with open(yaml_file_path, 'w') as f:
    yaml.dump(data_config, f)

print(f"Archivo {yaml_file_path} generado corectamente.")


Archivo /home/adrian/Desktop/Aprendizaje/practica_final/training/dataset/data.yaml generado corectamente.


## 3. Carga del Modelo y Transfer Learning

Cargamos un modelo YOLO pre-entrenado para segmentación.


In [3]:
# Al ejecutar esta celda por primera vez, el modelo se descargará automáticamente 
# desde los servidores de Ultralytics si no existe en la carpeta actual.
model = YOLO('yolo11s-seg.pt') 

# Entrenar el modelo (Fine-tuning)
# Descomenta la siguiente línea para iniciar el entrenamiento:
model.train(
    data=os.path.join(dataset_path, 'data.yaml'), 
    epochs=100, 
    imgsz=640, 
    batch=32,
    freeze=15,        # Congela las capas base para Few-Shot Learning
    mosaic=1.0,       # Aumento de datos agresivo
    mixup=0.5,        # Mezcla imágenes para mejorar generalización
    copy_paste=0.3,   # Ideal para segmentación de productos
    patience=20       # Detener si deja de mejorar
)

print("Modelo cargado y listo para entrenamiento.")


Ultralytics 8.4.8 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 7850MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/adrian/Desktop/Aprendizaje/practica_final/training/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=15, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.5, mode=train, model=yolo11s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train16, nbs=64, nms=False, opset=None, o

KeyboardInterrupt: 

## 4. Validación del Modelo

Después del entrenamiento, es importante evaluar el rendimiento del modelo utilizando el conjunto de datos de validación. Esto generará métricas como mAP (Precision Promedio Media).


In [ ]:
import glob
import re

# Validar el modelo con el set de validación definido en data.yaml
# Esto usará las imágenes en la carpeta 'valid/images' especificada en el archivo yaml
# Encontrar automáticamente el último entrenamiento (train con número más alto)
train_folders = glob.glob(os.path.join(base_dir, 'runs/segment/train*'))
if train_folders:
    # Extraer números de las carpetas train, train2, train3, etc.
    train_numbers = []
    for folder in train_folders:
        match = re.search(r'train(\d+)$', folder)
        if match:
            train_numbers.append((int(match.group(1)), folder))
        elif folder.endswith('train'):
            train_numbers.append((1, folder))
    
    # Ordenar y tomar el número más alto
    if train_numbers:
        latest_train = max(train_numbers, key=lambda x: x[0])[1]
        model_path = os.path.join(latest_train, 'weights/best.pt')
        print(f"Usando modelo entrenado más reciente: {model_path}")
    else:
        model_path = os.path.join(base_dir, 'runs/segment/train/weights/best.pt')
else:
    model_path = os.path.join(base_dir, 'runs/segment/train/weights/best.pt')

model = YOLO(model_path)
metrics = model.val()

# Mostrar resultados principales
print(f"\nMétricas de detección (Cajas):")
print(f" - mAP@50: {metrics.box.map50:.3f}")
print(f" - mAP@50-95: {metrics.box.map:.3f}")

if hasattr(metrics, 'seg') and metrics.seg is not None:
    print(f"\nMétricas de segmentación (Máscaras):")
    print(f" - mAP@50: {metrics.seg.map50:.3f}")
    print(f" - mAP@50-95: {metrics.seg.map:.3f}")


Usando modelo entrenado más reciente: /home/adrian/Desktop/Aprendizaje/practica_final/training/runs/segment/train13/weights/best.pt
Ultralytics 8.4.8 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 7850MiB)
YOLO11s-seg summary (fused): 114 layers, 10,068,364 parameters, 0 gradients, 32.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2254.6±894.4 MB/s, size: 25.5 KB)
val: Scanning /home/adrian/Desktop/Aprendizaje/practica_final/training/dataset/valid/labels... 4 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 4/4 1.9Kit/s 0.0s
val: New cache created: /home/adrian/Desktop/Aprendizaje/practica_final/training/dataset/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 7.9it/s 0.1s
                   all          4          4      0.665      0.167        0.1     0.0735      0.665      0.167        0.1      0.068
      SKU-

## 5. Solución Avanzada: Generación de Datos Sintéticos

Si tienes pocas imágenes (menos de 50 por producto), la mejor solución profesional es crear un dataset sintético. Esta celda de código te muestra cómo tomar UNA foto de tu producto (con fondo transparente o recortada) y pegarla sobre miles de fondos aleatorios. 

**Requisitos:**
1. Carpeta `backgrounds/` con fotos aleatorias (descarga un pack de internet).
2. Carpeta `objects/` con tus productos recortados (formato PNG transparente).

Esta técnica es la que usan las grandes empresas de Retail para entrenar sus sistemas.


In [ ]:
import random

def create_synthetic_image(background_path, object_path, output_path):
    """
    Función conceptual para generación de datos sintéticos.
    Toma un fondo y un objeto transparente, y los fusiona.
    """
    bg = cv2.imread(background_path)
    obj = cv2.imread(object_path, cv2.IMREAD_UNCHANGED) # Cargar con canal Alpha
    
    if obj.shape[2] != 4:
        print("El objeto debe ser PNG con transparencia (4 canales)")
        return

    # Redimensionar objeto aleatoriamente
    scale = random.uniform(0.3, 0.8)
    h, w = obj.shape[:2]
    new_size = (int(w*scale), int(h*scale))
    obj = cv2.resize(obj, new_size)
    
    # Calcular posición aleatoria
    # ... (lógica de mezcla alpha blending) ...
    # cv2.imwrite(output_path, list_img)
    print("Imagen sintética generada :)")

# Descomenta para usar cuando tengas tus recortes listos
create_synthetic_image('bgs/office.jpg', 'objects/laptop_sku1.png', 'train/images/syn_001.jpg')


## 5. Predicción

Una vez entrenado, podemos usar el modelo para predecir sobre nuevas imágenes.


In [ ]:
def predict_product(image_path):
    results = model.predict(source=image_path, save=True, show=False)
    
    for r in results:
        # Mostrar imagen con detecciones
        im_array = r.plot()  # plot a BGR numpy array of predictions
        plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

# predict_product('ruta/a/tu/imagen.jpg')


## 6. Prueba en Tiempo Real (Cámara)

Esta función te permite abrir la cámara de tu laptop para probar el modelo en tiempo real. Se abrirá una ventana emergente con el video procesado.


In [ ]:
import cv2
import numpy as np

def run_robust_camera(confidence_threshold=0.6):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error al abrir cámara")
        return

    # Definir un nombre de ventana único y fijo
    WINDOW_NAME = "Deteccion_YOLO_Segmentacion"
    cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)

    print(f"Cámara iniciada. Mostrando solo el objeto más probable.")
    print("Presiona 'q' para cerrar la ventana.")
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret: break
            
            # Copia para dibujar
            annotated_frame = frame.copy()
            
            # Inferencia YOLO
            # stream=True es más eficiente para video
            results = model(frame, conf=confidence_threshold, verbose=False, stream=False)
            
            # Tomamos el primer resultado (el frame actual)
            r = results[0]
            
            # Si hay detecciones en el frame
            if len(r.boxes) > 0:
                # YOLO ordena por confianza, el 0 es el mejor
                best_idx = 0
                box = r.boxes[best_idx]
                conf = float(box.conf[0])
                
                # Validar confianza
                if conf >= confidence_threshold:
                    # Clase y Coordenadas
                    cls_id = int(box.cls[0])
                    label = model.names[cls_id]
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    
                    # --- DIBUJAR SEGMENTACIÓN (La "selección" del objeto) ---
                    if r.masks is not None:
                        # Obtener polígono de la máscara
                        mask_coords = r.masks.xy[best_idx].astype(np.int32)
                        
                        # Crear un overlay semitransparente
                        overlay = annotated_frame.copy()
                        cv2.fillPoly(overlay, [mask_coords], (0, 255, 0)) # Relleno verde
                        # Mezclar original con overlay
                        cv2.addWeighted(overlay, 0.4, annotated_frame, 0.6, 0, annotated_frame)
                        # Dibujar contorno fino
                        cv2.polylines(annotated_frame, [mask_coords], True, (0, 255, 0), 2)
                    
                    # --- DIBUJAR CAJA Y TEXTO ---
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(annotated_frame, f"{label} {conf:.2f}", (x1, y1 - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            # --- CORRECCIÓN DE VENTANA ---
            # Mostramos el frame procesado en la misma ventana pre-configurada
            cv2.imshow(WINDOW_NAME, annotated_frame)
                
            # Salida con 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    except Exception as e:
        print(f"Ocurrió un error: {e}")
    finally:
        # Limpieza total para evitar ventanas colgadas
        cap.release()
        cv2.destroyAllWindows()
        # En entornos Linux/Jupyter, a veces se requiere waitKey extra para cerrar
        for i in range(5):
            cv2.waitKey(1)

# Iniciar cámara
run_robust_camera(confidence_threshold=0.50)


Cámara iniciada. Mostrando solo el objeto más probable.
Presiona 'q' para cerrar la ventana.


KeyboardInterrupt: 